In [5]:
# Setup
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import pandas as pd
from getpass import getpass

# Set the display options
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Connect to Neo4j
uri = "bolt://localhost:7687"
user = "neo4j"
password = getpass()

gds = GraphDataScience(uri, auth=(user,password))
gds.set_database("neo4j")

········


In [ ]:
# Create data with Cypher
gds.run_cypher("""
MERGE (mitch:Individual {name: 'Mitch'})

MERGE (carl:Individual {name: 'Carl'}),
       (nathan:Individual {name: 'Nathan'}),
       (sophia:Individual {name: 'Sophia'}),
       (emma:Individual {name: 'Emma'}),
       (olivia:Individual {name: 'Olivia'}),
       (james:Individual {name: 'James'}),
       (michael:Individual {name: 'Michael'}),
       (emily:Individual {name: 'Emily'}),
       (madison:Individual {name: 'Madison'}),
       (david:Individual {name: 'David'}),

       (mitch)-[:FRIEND_OF]->(carl),
       (mitch)-[:FRIEND_OF]->(sophia),
       (mitch)-[:FRIEND_OF]->(emma),
       (mitch)-[:FRIEND_OF]->(olivia),
       (mitch)-[:FRIEND_OF]->(james),
       (mitch)-[:FRIEND_OF]->(michael),
       (mitch)-[:FRIEND_OF]->(emily),
       (mitch)-[:FRIEND_OF]->(madison),
       (mitch)-[:FRIEND_OF]->(david),

       (carl)-[:FRIEND_OF]->(nathan);

""")

## Queries to Execute in Neo4j Browser

In [ ]:
// Return a single node based on WHERE clause.  Run in Browser, not Jupyter Notebook
MATCH (n1:Individual)
WHERE n1.name = 'Mitch'
RETURN *                        //return everything 
LIMIT 25
;

In [ ]:
// Return a collection of all Individuals matching this pattern
MATCH (n:Individual)
RETURN count(n) as Number_of_Individuals, collect(n.name) as Names
;

In [ ]:
MATCH (n1:Individual)-[r:FRIEND_OF]-(n2:Individual)
RETURN *

In [ ]:
MATCH (n1:Individual)-[r:FRIEND_OF]-(n2:Individual)
RETURN n1.name as source_name, type(r) as Relationship_Type, n2.name as target_name

## Switch to the Recipe Dataset

In [ ]:
:use recipe

In [ ]:
CREATE INDEX Recipe_Title IF NOT EXISTS FOR (n:Recipe) ON (n.Recipe_Title) 

In [ ]:
:show index

In [ ]:
MATCH (r:Recipe)
WHERE r.Recipe_Title CONTAINS 'cornbread'
RETURN *
LIMIT 100

In [ ]:
MATCH (r:Recipe)
WHERE r.Recipe_Title CONTAINS 'cornbread'
RETURN count(*) as cornbread_count

In [ ]:
MATCH (recipe)-[r:USES]->(i:Ingredient)
WHERE recipe.Recipe_Title CONTAINS 'cornbread'
RETURN recipe.Recipe_Title as Recipe_Title
, recipe.Directions as Directions
, collect(i.Ingredient_Name) as Ingredients
LIMIT 1

In [6]:
gds.set_database('recipe')

In [ ]:
gds.run_cypher(""" 
MATCH (recipe:Recipe)-[:USES]->(i:Ingredient)
WHERE i.Ingredient_Name IN ['almonds', 'walnuts']
    AND recipe.Recipe_Title CONTAINS 'spice'
WITH recipe, collect(i.Ingredient_Name) as ingredients
WHERE all(ingredient IN ['almonds', 'walnuts'] WHERE ingredient IN ingredients)
RETURN recipe.Recipe_Title as Recipe_Title, recipe.Directions as Directions, recipe.Link as Link
LIMIT 10
""")

In [ ]:
gds.run_cypher(""" 
MATCH (recipe)-[r:USES]->(i:Ingredient)
WHERE recipe.Recipe_Title CONTAINS 'cornbread'
    AND size(recipe.Directions) > 500
    AND i.Ingredient_Name <> 'sharp cheese'
RETURN recipe.Recipe_Title as Recipe_Title
, recipe.Directions as Directions
, collect(i.Ingredient_Name) as Ingredients
ORDER BY Recipe_Title
LIMIT 3
""")

In [ ]:
gds.run_cypher(""" 
MATCH (recipe)-[r:USES]->(i:Ingredient)
WHERE recipe.Recipe_Title CONTAINS 'cornbread'
WITH recipe, apoc.node.degree(recipe) as degree
SET recipe.degree = degree
""")

In [12]:
gds.run_cypher(""" 
MATCH (r1:Recipe)-[:USES]->(i:Ingredient)<-[:USES]-(r2:Recipe) 
WHERE r1.degree > 10 
    AND r2.degree > 10
    AND id(r1) < id(r2) 
RETURN count(distinct r1) as Recipe_Count
""")

,Recipe_Count
0,1228


In [7]:
gds.run_cypher(""" 
CALL gds.graph.project.cypher(
    'cornbreadGraph',
    'MATCH (r:Recipe) WHERE r.degree > 10 RETURN id(r) AS id',
    'MATCH (r1:Recipe)-[:USES]->(i:Ingredient)<-[:USES]-(r2:Recipe) 
     WHERE r1.degree > 10 AND r2.degree > 10 
     RETURN id(r1) AS source, id(r2) AS target'
);
""")

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
0,MATCH (r:Recipe) WHERE r.degree > 10 RETURN id(r) AS id,"MATCH (r1:Recipe)-[:USES]->(i:Ingredient)<-[:USES]-(r2:Recipe) \n WHERE r1.degree > 10 AND r2.degree > 10 \n RETURN id(r1) AS source, id(r2) AS target",cornbreadGraph,1230,3933770,6440


In [10]:
gds.run_cypher(""" 
CALL gds.nodeSimilarity.write('cornbreadGraph', {
    writeRelationshipType: 'SIMILAR_TO',
    writeProperty: 'similarity',
    similarityCutoff: 0.90
})
YIELD nodesCompared, relationshipsWritten;
""")

,nodesCompared,relationshipsWritten
0,1230,12300


In [4]:
gds.run_cypher(""" 
CALL gds.graph.drop('cornbreadGraph');
""")

Failed to write data to connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))


ServiceUnavailable: Failed to write data to connection IPv4Address(('localhost', 7687)) (IPv4Address(('127.0.0.1', 7687)))